In [2]:
import json, os
from datetime import datetime

# SPECIFY A FILE
json_files = [pos_json for pos_json in os.listdir('./json_files/') if pos_json.endswith('.json')]

# CONVERTING JSON TO QUESTION REQUIRING USER INPUT
f = open(os.path.join('./json_files','coe_redirect.json'))
file = f.read()
data = json.loads(file)

# get the title of form, and get corresponding identifier
form_title = data["metadata"]["title"]
print(form_title)

q_and_a = []

# Iterating over the form elements
elements = data["items"]
# Determining flow
redirect = False
target_index = None

# add a condition to exit loop and continue using a code
for e in elements:
    item_index = e["index"]
    if redirect == True:
        if item_index == target_index:
            redirect = False
        else:
            continue
    q_type = e['type']
    title = e["title"]
    # LIST is a dropdown
    if q_type == "LIST" or q_type =="MULTIPLE_CHOICE":
        print(title)
        choices = [x["choice"] for x in e["choices"]]           #travel, Credit Card Application
        nav_type = [x["navType"] for x in e["choices"]]         #GO_TO_PAGE, GO_TO_PAGE
        target_page = [x["targetIndex"] for x in e["choices"]]  #1, 5
        choices_lower = [x.lower() for x in choices]
        print(f"choices: {choices}")
        choice = None
        while choice is None:
             user_answer = input("please type your desired choice")
             if user_answer.lower() in choices_lower:               #nakapili na ng choice sa choices
                 choice = user_answer.lower()
                 choice_index = choices_lower.index(user_answer.lower()) #if travel is chosen, choice_index > 0
                 if nav_type[choice_index] == "GO_TO_PAGE":              #go_to_page
                     target_index = target_page[choice_index]            #1
                     redirect = True
             elif user_answer.lower() == 'exit':
                break
             else:
                print("{input} is not in the given choices".format(input=user_answer))
                continue
        q_and_a.append({"question":title,"answer":choice})    

    elif q_type == "CHECKBOX":
        print(title)
        choices = e["choices"]
        choices_lower = [x.lower() for x in choices]
        print(f"choices: {choices}")
        choice_list = []
        list_match = False
        while not list_match:
            user_answer = input("please type your desired choice/s separated by comma")
            answer_list = user_answer.split(",")
            answer_list_lower = [x.lower().strip() for x in answer_list]
            diff = set(answer_list_lower).difference(set(choices_lower))
            if user_answer.lower() == 'exit':
                break
            elif len(diff) == 0:
                list_match = True
                q_and_a.append({"question":title,"answer":answer_list_lower})
            else:
                print(f"{diff} not in choices")

    elif q_type == "DATE":
        # specify date format
        # sample format: "04-01-1995", dd-mm-yyyy
        print(title)
        format = "%d-%m-%Y"
        date_match = False
        while not date_match:
            user_answer = input("please type date in 'dd-mm-yyyy' format")
            answer = user_answer.strip()
            if answer.lower() == 'exit':
                break
            try:
                if bool(datetime.strptime(answer, format)):
                    q_and_a.append({"question":title,"answer":answer})
                    date_match = True
            except:
                print("{input} is not in the valid format".format(input=user_answer))

    elif q_type == "PAGE_BREAK":
        print(title)
        print(e["helpText"])

    elif q_type == "IMAGE":
        print("Image type, not yet implemented")
    elif q_type == "TEXT":
        text_val = input(title).strip()
        q_and_a.append({"question":title,"answer":text_val})
    else:
        print("Other type, not yet implemented")


# print(q_and_a)

Certificate of Employment
Purpose of Request
choices: ['Travel', 'Credit Card Application']


Details Needed

Please select applicable:
choices: ['Option A', 'Option B', 'Option C', 'Option D', 'Option E']
[{'question': 'Purpose of Request', 'answer': 'credit card application'}, {'question': 'Please select applicable:', 'answer': ['option e', 'option a']}]
